# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

**User will Need to have their own api_keys files for "from api_keys import gkey"

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
path = os.path.join('../','WeatherPy/output_data/','cities.csv')
cities_df=pd.read_csv(path, index_col=False)
#cities_df.head()
#cities_df.count()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

Create a heat map that displays the humidity for every city from Part I.

  ![heatmap](Images/heatmap.png)

In [4]:
#Configure gmaps.
gmaps.configure(api_key=gkey)
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "lng"]]  #.astype(float)
# Create a  Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=cities_df['Humidity'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

hotel_df=cities_df.loc[(cities_df['Max Temp']>70)&(cities_df['Max Temp']<80) & (cities_df['Wind Speed']<10) & (cities_df['Cloudiness']<10),:]
hotel_df

,City_Id,City,Cloudiness,Country,Date,Humidity,Lat,lng,Max Temp,Wind Speed
42,3460954,Itabirito,0,BR,1611877307,38,-20.2533,-43.8014,78.80,6.91
66,3374210,São Filipe,0,CV,1611876895,66,14.8961,-24.4956,73.29,5.35
103,1259385,Port Blair,4,IN,1611877187,84,11.6667,92.7500,78.60,7.92
110,1106677,Bambous Virieux,7,MU,1611877189,75,-20.3428,57.7575,78.75,7.61
123,2283016,Odienné,0,CI,1611877335,20,9.5100,-7.5692,72.81,2.93
152,1328421,Pathein,0,MM,1611877230,86,16.7833,94.7333,71.29,5.28
179,6690296,Saint-Joseph,0,RE,1611877356,69,-21.3667,55.6167,77.00,8.05
196,2189343,Kaeo,3,NZ,1611877231,45,-35.1000,173.7833,70.50,9.08
216,2435124,Bitkine,9,TD,1611877368,19,11.9801,18.2138,70.21,9.13
223,2277060,Gbarnga,7,LR,1611877371,98,6.9956,-9.4722,71.60,1.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df.head()

<ipython-input-6-06096619966f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City_Id,City,Cloudiness,Country,Date,Humidity,Lat,lng,Max Temp,Wind Speed,Hotel Name
42,3460954,Itabirito,0,BR,1611877307,38,-20.2533,-43.8014,78.80,6.91,
66,3374210,São Filipe,0,CV,1611876895,66,14.8961,-24.4956,73.29,5.35,
103,1259385,Port Blair,4,IN,1611877187,84,11.6667,92.7500,78.60,7.92,
110,1106677,Bambous Virieux,7,MU,1611877189,75,-20.3428,57.7575,78.75,7.61,
123,2283016,Odienné,0,CI,1611877335,20,9.5100,-7.5692,72.81,2.93,


In [7]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
# target_coordinates = ideal_df['Lat'], ideal_df['lng']
# target_search = ideal_df['City']
# target_radius = 5000
# #target_type = "hotel"
# set up a parameters dictionary

for index, row in hotel_df.iterrows():
  
    target_coordinates = (f'{row["Lat"]},{row["lng"]}')
    #target_search = row['City']
    target_radius = 5000
    target_type = "lodging"
    
    try:
        params = {
            "location": target_coordinates,
            #"keyword": target_search,
            "radius": target_radius,
            "type": target_type,
            "key": gkey
        }

        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)
        response=response.json()
        hotel_df.loc[index, "Hotel Name"]=response["results"][0]["name"]
      
    except:
        print(f'Issue with {index}')


C:\Users\cpoir\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Issue with 216
Issue with 549


In [8]:
hotel_df

,City_Id,City,Cloudiness,Country,Date,Humidity,Lat,lng,Max Temp,Wind Speed,Hotel Name
42,3460954,Itabirito,0,BR,1611877307,38,-20.2533,-43.8014,78.80,6.91,Aredes Apart Hotel
66,3374210,São Filipe,0,CV,1611876895,66,14.8961,-24.4956,73.29,5.35,Tortuga B&B
103,1259385,Port Blair,4,IN,1611877187,84,11.6667,92.7500,78.60,7.92,Welcomhotel Bay Island Port Blair
110,1106677,Bambous Virieux,7,MU,1611877189,75,-20.3428,57.7575,78.75,7.61,Casa Tia Villa
123,2283016,Odienné,0,CI,1611877335,20,9.5100,-7.5692,72.81,2.93,Oustaz Vame DIAKITE
152,1328421,Pathein,0,MM,1611877230,86,16.7833,94.7333,71.29,5.28,Pathein Hotel
179,6690296,Saint-Joseph,0,RE,1611877356,69,-21.3667,55.6167,77.00,8.05,"""Plantation Bed and Breakfast"
196,2189343,Kaeo,3,NZ,1611877231,45,-35.1000,173.7833,70.50,9.08,Bushido Lodge
216,2435124,Bitkine,9,TD,1611877368,19,11.9801,18.2138,70.21,9.13,
223,2277060,Gbarnga,7,LR,1611877371,98,6.9956,-9.4722,71.60,1.59,Passion Hotel 2


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "lng"]]


In [10]:
#Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.
# Add marker layer ontop of heat map

#markers=gmaps.marker_layer(locations)
#fig.add_layer(markers)    #these two lines work by themselves. Trying to add Hotel names below
#markers=gmaps.marker_layer(locations)


markers=gmaps.marker_layer(
    locations,
    #hover_text=hotel_info  # not working
    info_box_content=hotel_info
)

fig.add_layer(markers) 
fig


Figure(layout=FigureLayout(height='420px'))